In [35]:
import warnings
warnings.simplefilter("ignore")
from sys import exit
import os.path
import cv2
import numpy as np
import pandas as pd
from tkinter.filedialog import askdirectory
from scipy import ndimage
from skimage import measure, io, exposure, morphology
import matplotlib.pyplot as plt

In [50]:
'''
Define pixel size of current camera in nm/pix-el: MD: 114, Andor: 128, Sam: 101.4
'''
pixelSize = 128
threshold_value = 70
footprint = np.ones([5,5])

In [46]:
def findBackground(img):
    #avgz_fit = np.zeros(img.shape, np.float64)
    len_x = img.shape[0]
    len_y = img.shape[1]
    z = img.flatten()
    z = z.reshape((len(z),1))
    y = np.array([ i for i in range(len_y) ]*len_x)
    x = np.array([ [i]*len_y for i in range(len_x)  ])
    x = x.flatten()
    xx = np.power(x, 2)
    yy = np.power(y, 2)
    xy = x*y
    ones = np.ones(len(z))
    H = np.array([ xx, yy, xy, x, y, ones ])
    H = H.T
    c = np.dot(H.T, H)
    c = np.linalg.inv(c) 
    c = np.dot(c, H.T)
    c = np.dot(c, z)
    z_fit = np.dot(H, c)
    z_fit = z_fit.reshape((len_x, len_y)).astype(np.uint16)
    return(z_fit)

In [47]:
def findObjects(img, thrval):
    mean = np.mean(img)
    saliency = np.power(img - mean,2)
    dx = ndimage.sobel(saliency, 0)  # horizontal derivative
    dy = ndimage.sobel(saliency, 1)  # vertical derivative
    mag = np.hypot(dx, dy)  # magnitude
    mag *= 255 / np.max(mag)  # normalize
    binary_mask = mag >= thrval
    clean_mask = ndimage.morphology.binary_fill_holes(binary_mask).astype(np.uint8)
    return(clean_mask,mag.astype(np.uint8),binary_mask.astype(np.uint8))

In [56]:
'''Ask for picture location'''
print("\n please choose video folder!")
dirtouse = askdirectory(title = "Select foldername", initialdir = '/home/edwin/Desktop/testimg/')

tifs = []
imgs = []

'''Move through all files in directory, saving file names and images if they're tifs'''
for filename in sorted(os.listdir(dirtouse)):
    if filename.endswith('.tif'):
        tifs.append(filename)
        pictouse = os.path.join(dirtouse,filename)
        img = io.imread(pictouse,-1)
        imgs.append(img)
length = len(imgs) - 1
allpanalysis = pd.DataFrame()
'''Move through all imgs'''
for i in range(len(imgs)):
    print(str(i+1)+'/'+str(length+1))
    img = imgs[i]
    eimg = exposure.equalize_adapthist(img)
    imgname = tifs[i][:-4]
    bgd = findBackground(img).astype(np.uint16)
    bgdImg = cv2.subtract(img, bgd)
    gbgdImg = ndimage.filters.gaussian_filter(bgdImg,1)
    clean_mask,mag_img,dirty_mask = findObjects(gbgdImg,threshold_value)
    super_clean_mask = morphology.binary_closing(clean_mask, footprint=footprint)
    label_img = measure.label(super_clean_mask)
    props = measure.regionprops_table(label_img,intensity_image=img,
                                        properties=['label','area', 'bbox', 'bbox_area', 'convex_area',
                                                    'centroid', 'weighted_centroid', 'weighted_local_centroid', 
                                                    'eccentricity', 'equivalent_diameter', 
                                                    'inertia_tensor', 'inertia_tensor_eigvals', 
                                                    'local_centroid', 'major_axis_length', 
                                                    'max_intensity', 'mean_intensity', 'min_intensity', 
                                                    'minor_axis_length', 'solidity',  'orientation', 'perimeter', 
                                                    'moments', 'moments_central', 'moments_hu', 
                                                    'moments_normalized', 'weighted_moments', 'weighted_moments_central', 
                                                    'weighted_moments_hu', 'weighted_moments_normalized'])

    panalysis = pd.DataFrame(props)
    panalysis.to_csv(os.path.join(dirtouse,imgname+'.csv'))
    panalysis['imagename']= imgname
    allpanalysis = pd.concat([allpanalysis,panalysis.dropna(axis=1)])
allpanalysis.index=np.arange(0,len(allpanalysis))
allpanalysis.to_csv(os.path.join(dirtouse,'totalanalysis.csv'))


 please choose video folder!
1/3
2/3
3/3
